In [1]:
%matplotlib inline
import matplotlib.pyplot as plt 
import numpy as np 
import cv2
from scipy.signal import find_peaks
from skimage import img_as_float

## Threshold

In [2]:
M = cv2.cvtColor(cv2.imread('Chanel_5.jpg'), cv2.COLOR_BGR2RGB)
R,G,B = cv2.split(M)

M_gray = cv2.cvtColor(M, cv2.COLOR_RGB2GRAY)


# plt.figure(figsize=(15, 10))
# plt.axis('off')
# plt.imshow(M)
# plt.title('Original Image')
# plt.show()

plt.figure(figsize=[15,10])
plt.axis('off')
plt.imshow(M_gray,'gray')
#plt.title('Original Image in Grayscale')
plt.show()

error: OpenCV(4.1.0) C:\projects\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [ ]:
hist, bin_edges = np.histogram(M_gray, bins = np.linspace(0,255,257), density = "True")
norm_hist = (hist-min(hist))/(max(hist)-min(hist))

plt.figure(figsize=[8,5])
plt.rcParams['axes.titlesize'] = 17
plt.rcParams.update({'font.size': 15})
plt.plot(hist,'turquoise',linewidth=2.5)

plt.xlabel('Pixel values')
plt.ylabel('Intensity')
plt.title('Original Histogram')
plt.show()

plt.figure(figsize=[8,5])
plt.rcParams['axes.titlesize'] = 17
plt.rcParams.update({'font.size': 15})
plt.plot(norm_hist,'turquoise',linewidth=2.5)
plt.xlabel('Pixel values')
plt.ylabel('Intensity')
plt.title('Normalized Histogram (PDF)')
plt.show()



In [ ]:
peak, _ = find_peaks(norm_hist,height=0.02)


plt.figure(figsize=[8,5])
plt.rcParams['axes.titlesize'] = 17
plt.rcParams.update({'font.size': 15})
plt.plot(norm_hist,'turquoise',linewidth=2.5)
plt.plot(peak, norm_hist[peak],'x',  color='r', markersize = 10, linewidth=6,label='Peak value')
#plt.text(peak, norm_hist[peak],peak,ha='left', va='top')
plt.legend()
plt.xlabel('Pixel values')
plt.ylabel('Intensity')
plt.title('Normalized Histogram (PDF)')
plt.show()

print(peak)

# Thresholding Method

In [ ]:
BW = M_gray < 220

plt.figure(figsize=[15,6])
plt.rcParams['axes.titlesize'] = 17
plt.subplot(121)
plt.imshow(M_gray,'gray')
plt.title('Grayscale Image')
plt.axis('off')
plt.subplot(122)
plt.imshow(BW,'gray')
plt.title('Segmented Image')
plt.axis('off')
plt.show()

## NCC

In [ ]:
# M_sub = cv2.cvtColor(cv2.imread('Patch_Chanel_Bag.png'), cv2.COLOR_BGR2RGB)

# hist_sub, bin_edges_sub = np.histogram(M_sub, bins = np.linspace(0,255,257), density = "True")
# norm_hist_sub = (hist_sub-min(hist_sub))/(max(hist_sub)-min(hist_sub))

# M_f = img_as_float(M) 
# R,G,B = cv2.split(M_f)

# # Overall pixels
# I = R + G + B
# r = R/I
# g = G/I
# b = 1-r-g

# # subregion
# M_f_sub = img_as_float(M_sub) 
# R1,G1,B1 = cv2.split(M_f_sub)

# I_s = R1 + G1 + B1
# r_s = R1/I_s
# g_s = G1/I_s
# b_s = 1-r_s-g_s

# # plt.figure(figsize=[12,5])
# # plt.rcParams['axes.titlesize'] = 17
# # plt.imshow(M_sub)
# # plt.title('Subregion of ROI')
# # plt.axis('off')
# # plt.show()

# plt.figure(figsize=[8,5])
# plt.rcParams['axes.titlesize'] = 17
# plt.rcParams.update({'font.size': 15})
# plt.plot(norm_hist_sub,'turquoise',linewidth=2.5)
# plt.xlabel('Pixel values')
# plt.ylabel('Intensity')
# plt.title('Normalized Histogram (PDF) of Subregion of ROI')
# plt.show()

# Parametric Segmentation

In [ ]:
# Gaussian Function
def Gauss_dist(c,c_s):
    cc = 1/(np.std(c_s)*np.sqrt(2*np.pi))
    cep = -(c-np.mean(c_s))**2/(2*np.std(c_s)**2)
    pc = cc*np.exp(cep)
    return pc

#Parametric
def Parametric(image, patch):
    M = cv2.cvtColor(cv2.imread(image), cv2.COLOR_BGR2RGB)
    M_sub = cv2.cvtColor(cv2.imread(patch), cv2.COLOR_BGR2RGB)

    hist_sub, bin_edges_sub = np.histogram(M_sub, bins = np.linspace(0,255,257), density = "True")
    norm_hist_sub = (hist_sub-min(hist_sub))/(max(hist_sub)-min(hist_sub))

    M_f = img_as_float(M) 
    R,G,B = cv2.split(M_f)

    # Overall pixels
    I = R + G + B
    r = R/I
    g = G/I
    b = 1-r-g

    # subregion
    M_f_sub = img_as_float(M_sub) 
    R1,G1,B1 = cv2.split(M_f_sub)

    I_s = R1 + G1 + B1
    r_s = R1/I_s
    g_s = G1/I_s
    b_s = 1-r_s-g_s
    
    
    #Parametric Segmentation Part
    pr = Gauss_dist(r,r_s)
    pg = Gauss_dist(g,g_s)
    joint = pr*pg

    plt.figure(figsize=[8,5])
    plt.rcParams['axes.titlesize'] = 17
    plt.rcParams.update({'font.size': 15})
    plt.plot(joint,'turquoise',linewidth=2.5)
    plt.xlabel('Pixel values')
    plt.ylabel('Intensity')
    plt.title('Joint Probability Distribution of the Image')
    plt.show()


    plt.figure(figsize=[15,6])
    plt.rcParams['axes.titlesize'] = 17
    plt.imshow(joint,'gray')
    plt.title('Result Image')
    plt.axis('off')
    plt.show()

## R vs. G

In [ ]:
image = M
cropped = M_sub

image = image.astype(float)
cropped = cropped.astype(float)

R_image = image[:,:,0]
G_image = image[:,:,1]
B_image = image[:,:,2]

R_cropped = cropped[:,:,0]
G_cropped = cropped[:,:,1]
B_cropped = cropped[:,:,2]

I_image = R_image + G_image + B_image
I_cropped = R_cropped + G_cropped + B_cropped

In [ ]:
I_image[I_image==0] = 10000000
I_cropped[I_cropped==0] = 10000000

r_image = R_image/I_image
g_image = G_image/I_image
b_image = B_image/I_image

r_cropped = R_cropped/I_cropped
g_cropped = G_cropped/I_cropped
b_cropped = B_cropped/I_cropped

b_image = 1 - r_image - g_image
b_cropped = 1 - r_cropped - g_cropped

In [ ]:
plt.scatter(r_image,g_image)
plt.xlim(0,1)
plt.ylim(0,1)
plt.show() 

In [ ]:
plt.scatter(r_cropped,g_cropped)
plt.xlim(0,1)
plt.ylim(0,1)
plt.show()

## Non Parametric

In [ ]:
def Non_parametric(image,patch): 
    
    roi = cv2.imread(patch)
    hsv = cv2.cvtColor(roi,cv2.COLOR_BGR2HSV)

    target = cv2.imread(image)
    hsvt = cv2.cvtColor(target,cv2.COLOR_BGR2HSV)


    # calculating object histogram
    roihist = cv2.calcHist([hsv],[0, 1], None, [1, 256], [0, 180, 0, 256] )
   
    # normalize histogram and apply backprojection
    cv2.normalize(roihist,roihist,0,255,cv2.NORM_MINMAX)
    dst = cv2.calcBackProject([hsvt],[0,1],roihist,[0,180,0,256],1)

    # Now convolute with circular disc
    disc = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5))
    cv2.filter2D(dst,-1,disc,dst)

    # threshold and binary AND
    ret,thresh = cv2.threshold(dst,50,255,0)
    thresh = cv2.merge((thresh,thresh,thresh))
    res = cv2.bitwise_and(target,thresh)
    res_1 = np.vstack((target,thresh,res))

    res[res == 0]= 255
    #a = res[:,:,::-1]
    cv2.imwrite('res.jpg', res_1)

## Results

In [ ]:
Non_parametric('Chanel_5.jpg','Chanel_5_Patch.png')

In [ ]:
Parametric('Chanel_5.jpg','Chanel_5_Patch.png')